In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Psoriatic_Arthritis"
cohort = "GSE57383"

# Input paths
in_trait_dir = "../../input/GEO/Psoriatic_Arthritis"
in_cohort_dir = "../../input/GEO/Psoriatic_Arthritis/GSE57383"

# Output paths
out_data_file = "../../output/preprocess/Psoriatic_Arthritis/GSE57383.csv"
out_gene_data_file = "../../output/preprocess/Psoriatic_Arthritis/gene_data/GSE57383.csv"
out_clinical_data_file = "../../output/preprocess/Psoriatic_Arthritis/clinical_data/GSE57383.csv"
json_path = "../../output/preprocess/Psoriatic_Arthritis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression of CD14+ cells from RA, PsA and PsO patients with Infliximab treatment"
!Series_summary	"objection: The immune inflammatory disorders rheumatoid arthritis (RA), psoriatic arthritis (PsA) and psoriasis (Ps) share common pathologic features and show responsiveness to anti-tumor necrosis factor (TNF) agents yet they are phenotypically distinct. The aim of this study was to examine if anti-TNF therapy is associated with divergent gene expression profiles in circulating cells and target tissues of patients with these diseases"
!Series_summary	"Method: Peripheral blood CD14+ and CD14- cells were isolated from 9 RA, 12 PsA and 10 Ps patients before and after infliximab (IFX) treatment"
!Series_overall_design	"Between April 2007 and June 2009, 31 patients with active RA, PsA and Ps who were naïve to anti-TNF agents, were recruited from the Faculty Rheumatology Clinics at the University of Rochester Medical Center after informed, written co

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the background information, this dataset appears to contain gene expression data from CD14+ cells,
# not just miRNA or methylation data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait - the disease/control status can be found in key 6 of the sample characteristics
trait_row = 6

# For age - age information is in key 2
age_row = 2

# For gender - sex information is in key 1
gender_row = 1

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert disease status to binary (0 for control, 1 for Psoriatic Arthritis)"""
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Psoriatic Arthritis is our trait of interest
    if value == "Psoriatic Arthritis":
        return 1
    elif value == "Health Control":
        return 0
    else:
        # Other diseases (Psoriasis, Rheumatoid Arthritis) are not our target trait
        return None

def convert_age(value):
    """Convert age string to integer value"""
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return int(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender to binary (0 for female, 1 for male)"""
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.upper() == 'F':
        return 0
    elif value.upper() == 'M':
        return 1
    else:
        return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted features
    print("Preview of clinical data:")
    print(preview_df(clinical_df))
    
    # Create the directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of clinical data:
{'GSM1381524': [0.0, 40.0, 0.0], 'GSM1381525': [0.0, 40.0, 0.0], 'GSM1381526': [0.0, 54.0, 1.0], 'GSM1381527': [0.0, 36.0, 0.0], 'GSM1381528': [0.0, 23.0, 0.0], 'GSM1381529': [0.0, 42.0, 0.0], 'GSM1381530': [0.0, 24.0, 0.0], 'GSM1381531': [0.0, 23.0, 1.0], 'GSM1381532': [0.0, 62.0, 1.0], 'GSM1381533': [0.0, 46.0, 1.0], 'GSM1381534': [0.0, 56.0, 0.0], 'GSM1381535': [0.0, 32.0, 0.0], 'GSM1381536': [0.0, 47.0, 0.0], 'GSM1381537': [0.0, 60.0, 0.0], 'GSM1381538': [0.0, 44.0, 0.0], 'GSM1381539': [0.0, 46.0, 0.0], 'GSM1381540': [0.0, 36.0, 0.0], 'GSM1381541': [0.0, 64.0, 1.0], 'GSM1381542': [0.0, 23.0, 1.0], 'GSM1381543': [nan, 19.0, 0.0], 'GSM1381544': [nan, 24.0, 0.0], 'GSM1381545': [nan, 46.0, 1.0], 'GSM1381546': [nan, 61.0, 1.0], 'GSM1381547': [nan, 57.0, 1.0], 'GSM1381548': [nan, 57.0, 1.0], 'GSM1381549': [nan, 35.0, 0.0], 'GSM1381550': [nan, 53.0, 1.0], 'GSM1381551': [nan, 59.0, 0.0], 'GSM1381552': [nan, 53.0, 1.0], 'GSM1381553': [nan, 44.0, 0.0], 'GSM1381554':

### Step 3: Gene Data Extraction

In [4]:
# 1. Use the get_genetic_data function from the library to get the gene_data from the matrix_file previously defined.
gene_data = get_genetic_data(matrix_file)

# 2. Print the first 20 row IDs (gene or probe identifiers) for future observation.
print(gene_data.index[:20])


Index(['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at',
       '1294_PM_at', '1316_PM_at', '1320_PM_at', '1405_PM_i_at', '1431_PM_at',
       '1438_PM_at', '1487_PM_at', '1494_PM_f_at', '1552256_PM_a_at',
       '1552257_PM_a_at', '1552258_PM_at', '1552261_PM_at', '1552263_PM_at',
       '1552264_PM_a_at', '1552266_PM_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers follow the Affymetrix probe ID format (e.g., '1007_PM_s_at', '1053_PM_at')
# These are microarray probe identifiers from the Affymetrix platform, not standard human gene symbols
# They need to be mapped to standard gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1007_PM_s_at', '1053_PM_at', '117_PM_at', '121_PM_at', '1255_PM_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010', 'Aug 20, 2010'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /DEFINITION=HSP70B Human heat-shock protein HSP70B' gene", 'X

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in the gene annotation dataframe correspond to gene identifiers and gene symbols
# By examining the preview, 'ID' column contains probe IDs matching those in gene_data
# 'Gene Symbol' column contains the corresponding gene symbols
prob_col = 'ID'
gene_col = 'Gene Symbol'

# 2. Get a gene mapping dataframe by extracting these two columns
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)

# 3. Convert probe-level measurements to gene expression data
# This function handles dividing probe expression when mapping to multiple genes
# and sums up all probe contributions for each gene
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print first few rows to verify the mapping was successful
print("First few rows of mapped gene expression data:")
print(gene_data.head())


First few rows of mapped gene expression data:
       GSM1381524  GSM1381525  GSM1381526  GSM1381527  GSM1381528  GSM1381529  \
Gene                                                                            
A1BG       5.7818      6.2652      5.9438      6.0062      6.2384      5.8118   
A1CF       4.0327      4.0811      4.2793      4.3403      4.2245      3.9326   
A2BP1     12.2527     11.7734     12.2891     11.7696     12.4713     11.7472   
A2LD1      8.3018      9.2080      9.2638      8.4592      9.0925      8.5653   
A2M        9.2125      9.2819      9.6747      9.4160     11.6454     10.0736   

       GSM1381530  GSM1381531  GSM1381532  GSM1381533  ...  GSM1381626  \
Gene                                                   ...               
A1BG       5.7488      6.5247      5.8682      6.3062  ...      5.5508   
A1CF       4.3102      4.7254      4.0183      4.0832  ...      4.0731   
A2BP1     12.0019     13.4765     12.4701     12.3507  ...     12.2121   
A2LD1      7.96

### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize the obtained gene data with the 'normalize_gene_symbols_in_index' function from the library.
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# Re-extract clinical features since we need it for linking
selected_clinical_df = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# 2. Link the clinical and genetic data with the 'geo_link_clinical_genetic_data' function from the library.
linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
print(f"Linked data shape before handling missing values: {linked_data.shape}")

# 3. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 4. Determine whether the trait and some demographic features are severely biased, and remove biased features.
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 5. Conduct quality check and save the cohort information.
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=True, 
    is_trait_available=True, 
    is_biased=is_trait_biased, 
    df=unbiased_linked_data,
    note="Dataset contains gene expression from endothelial cells derived from circulating progenitors of RA patients"
)

# 6. If the linked data is usable, save it as a CSV file to 'out_data_file'.
if is_usable:
    print(f"Data is usable. Saving to {out_data_file}")
    unbiased_linked_data.to_csv(out_data_file)
else:
    print("Data is not usable. Not saving linked data file.")

Gene data saved to ../../output/preprocess/Psoriatic_Arthritis/gene_data/GSE57383.csv
Linked data shape before handling missing values: (112, 18625)


Linked data shape after handling missing values: (55, 18625)
For the feature 'Psoriatic_Arthritis', the least common label is '0.0' with 19 occurrences. This represents 34.55% of the dataset.
The distribution of the feature 'Psoriatic_Arthritis' in this dataset is fine.

Quartiles for 'Age':
  25%: 39.0
  50% (Median): 46.0
  75%: 60.0
Min: 23.0
Max: 70.0
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 24 occurrences. This represents 43.64% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.

Data is usable. Saving to ../../output/preprocess/Psoriatic_Arthritis/GSE57383.csv
